In [1]:
import numpy as np
import pandas as pd

In [2]:
#读取数据
train = pd.read_csv('./data/yun_data/train_set.csv',sep='\t')

In [3]:
#查看数据基本情况以及分布
train.head(),train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   label   200000 non-null  int64 
 1   text    200000 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.1+ MB


(   label                                               text
 0      2  2967 6758 339 2021 1854 3731 4109 3792 4149 15...
 1     11  4464 486 6352 5619 2465 4802 1452 3137 5778 54...
 2      3  7346 4068 5074 3747 5681 6093 1777 2226 7354 6...
 3      2  7159 948 4866 2109 5520 2490 211 3956 5520 549...
 4      3  3646 3055 3055 2490 4659 6065 3370 5814 2465 5...,
 None)

In [4]:
#label标签的分布情况
# {'科技': 0, '股票': 1, '体育': 2, '娱乐': 3, '时政': 4, '社会': 5, '教育': 6, '财经': 7, '家居': 8, '游戏': 9, '房产': 10, '时尚': 11, '彩票': 12, '星座': 13}
train.label.value_counts(),train.label.value_counts(1)
#样本分布不均，0类别接近4W，13类别才900个左右

(label
 0     38918
 1     36945
 2     31425
 3     22133
 4     15016
 5     12232
 6      9985
 7      8841
 8      7847
 9      5878
 10     4920
 11     3131
 12     1821
 13      908
 Name: count, dtype: int64,
 label
 0     0.194590
 1     0.184725
 2     0.157125
 3     0.110665
 4     0.075080
 5     0.061160
 6     0.049925
 7     0.044205
 8     0.039235
 9     0.029390
 10    0.024600
 11    0.015655
 12    0.009105
 13    0.004540
 Name: proportion, dtype: float64)

In [5]:
train['text_split_len'] = train['text'].apply(lambda x: len(x.split()))
train['text_split_len'].max(), train['text_split_len'].min()

(57921, 2)

In [6]:
train['text_split_len'].describe()
# 赛题中每个新闻包含的字符个数平均为900个，还有一些新闻字符较长，以及太短，只有2个字符

count    200000.000000
mean        907.207110
std         996.029036
min           2.000000
25%         374.000000
50%         676.000000
75%        1131.000000
max       57921.000000
Name: text_split_len, dtype: float64

In [7]:
#字符分布统计 看看那个字出现最多
from collections import Counter
all_lines = ' '.join(list(train['text']))
word_count = Counter(all_lines.split(' '))
word_count = sorted(word_count.items(), key=lambda d:d[1], reverse = True)

In [8]:
#总共字符 6869种字符
print(len(word_count))
#最多
print(word_count[0])
#最少
print(word_count[-1])

6869
('3750', 7482224)
('3133', 1)


In [9]:
#这里还可以根据字在每个句子的出现情况，反推出标点符号。下面代码统计了不同字符在句子中出现的次数，其中字符3750，字符900和字符648在20w新闻的覆盖率接近99%，很有可能是标点符号。
train['text_unique'] = train['text'].apply(lambda x: ' '.join(set(x.split(' '))))
all_lines = ' '.join(list(train['text_unique']))
word_count = Counter(all_lines.split(" "))
word_count = sorted(word_count.items(), key=lambda d:int(d[1]), reverse = True)

In [21]:
#前一部分的字符，可能都是些标点符号
for word in word_count[0:11]:
    print(word, word[1]/len(train)*100.)

('3750', 197997) 98.9985
('900', 197653) 98.8265
('648', 191975) 95.9875
('2465', 177310) 88.655
('6122', 176543) 88.2715
('7399', 176249) 88.1245
('4811', 169371) 84.6855
('4464', 167161) 83.5805
('1699', 164856) 82.428
('3659', 163171) 81.5855
('2400', 163070) 81.535


In [ ]:
# 赛题中每个新闻包含的字符个数平均为1000个，还有一些新闻字符较长；
# 赛题中新闻类别分布不均匀，科技类新闻样本量接近4w，星座类新闻样本量不到1k；
# 赛题总共包括7000-8000个字符；
# 通过数据分析，我们还可以得出以下结论：
#
# 每个新闻平均字符个数较多，可能需要截断；
#
# 由于类别不均衡，会严重影响模型的精度；

In [34]:
#统计每类新闻中出现次数对多的字符
for label in sorted(train['label'].unique()):
    all_lines = ' '.join(list(train[train.label==label]['text']))
    word_count = Counter(all_lines.split(' '))
    word_count = sorted(word_count.items(), key=lambda d:d[1], reverse = True)
    #前一部分的字符，可能都是些标点符号
    print(f'---------------这是标签{label}的TOP10字符-------------')
    for word in word_count[0:11]:
        if word[0] not in ['3750','648','900']:
            print(word)

---------------这是标签0的TOP10字符-------------
('3370', 503768)
('4464', 307431)
('2465', 296582)
('6122', 272185)
('3659', 261338)
('7399', 260317)
('4939', 239933)
('4559', 209894)
---------------这是标签1的TOP10字符-------------
('3370', 626708)
('4464', 445525)
('3659', 388742)
('5036', 321769)
('6250', 275931)
('1633', 248565)
('6065', 224978)
('6357', 219363)
---------------这是标签2的TOP10字符-------------
('7399', 351894)
('6122', 343850)
('4939', 337762)
('4704', 305488)
('1667', 301478)
('5598', 287552)
('669', 246758)
('1099', 243205)
---------------这是标签3的TOP10字符-------------
('6122', 187933)
('4939', 173606)
('4893', 148784)
('7399', 147338)
('669', 146042)
('803', 136564)
('1635', 136345)
('5598', 129091)
---------------这是标签4的TOP10字符-------------
('4411', 120442)
('7399', 86190)
('4893', 77475)
('6122', 75635)
('4464', 69311)
('2400', 68390)
('4853', 64847)
('3370', 64757)
---------------这是标签5的TOP10字符-------------
('6122', 159125)
('5598', 136713)
('4893', 130570)
('7399', 122535)
('4939', 1